# Forecasting with Covariates

Improve forecast accuracy by incorporating external variables (covariates) that influence your time series.

**Topics:**
- Types of covariates (static vs dynamic, categorical vs numerical)
- Preparing covariate data
- Comparing forecasts with and without covariates
- XReg modes for combining covariates with time series patterns

## Understanding Covariates

Covariates are external variables that help explain patterns in your time series:

| Type | Static | Dynamic |
|------|--------|--------|
| **Categorical** | Country, product category | Day of week, holiday flag |
| **Numerical** | Store size, base price | Temperature, ad spend |

**Key rule:** Dynamic covariates must include values for both history AND forecast horizon.

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from gradientcast import GradientCastFM

# Replace with your API key
GRADIENTCAST_API_KEY = "your-api-key-here"

fm = GradientCastFM(api_key=GRADIENTCAST_API_KEY)

---
## Dataset: Electricity Prices

We'll use hourly electricity prices for France and Belgium, with covariates:
- `gen_forecast`: Generation forecast (dynamic numerical)
- `week_day`: Day of week (dynamic categorical)
- `country`: Country code (static categorical)

In [ ]:
# Load public electricity price dataset
df = pd.read_csv('https://datasets-nixtla.s3.amazonaws.com/EPF_FR_BE.csv')
df['ds'] = pd.to_datetime(df['ds'])

print(f"Dataset: {len(df):,} rows")
print(f"Columns: {list(df.columns)}")
print(f"Countries: {df['unique_id'].unique()}")
df.head()

In [ ]:
# Visualize the data
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

for ax, country in zip(axes, ['FR', 'BE']):
    country_df = df[df['unique_id'] == country].iloc[:500]  # First 500 hours
    ax.plot(country_df['ds'], country_df['y'], label='Price')
    ax.set_title(f'{country} Electricity Prices')
    ax.set_ylabel('Price')
    ax.legend()
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

---
## Preparing Data

Let's prepare a forecast window: 120 hours of history → 24 hours ahead.

In [ ]:
CONTEXT_LEN = 120  # 5 days of history
HORIZON_LEN = 24   # 1 day forecast

def prepare_batch(df, countries=['FR', 'BE'], start_idx=0):
    """Prepare a batch of data for forecasting."""
    input_data = {}
    gen_forecast = {}
    week_day = {}
    country_map = {}
    actuals = {}
    
    for country in countries:
        sub_df = df[df['unique_id'] == country].reset_index(drop=True)
        end_idx = start_idx + CONTEXT_LEN
        
        # Historical data
        input_data[country] = sub_df['y'][start_idx:end_idx].tolist()
        
        # Dynamic covariates: history + horizon
        gen_forecast[country] = sub_df['gen_forecast'][start_idx:end_idx + HORIZON_LEN].tolist()
        week_day[country] = sub_df['week_day'][start_idx:end_idx + HORIZON_LEN].astype(str).tolist()
        
        # Static covariate
        country_map[country] = country
        
        # Actual values for evaluation
        actuals[country] = sub_df['y'][end_idx:end_idx + HORIZON_LEN].tolist()
    
    return input_data, gen_forecast, week_day, country_map, actuals

# Prepare sample batch
input_data, gen_forecast, week_day, country_map, actuals = prepare_batch(df)

print(f"Input data length: {len(input_data['FR'])} hours")
print(f"Dynamic covariate length: {len(gen_forecast['FR'])} hours (context + horizon)")

---
## Baseline: Forecast Without Covariates

In [ ]:
# Forecast without covariates
base_result = fm.forecast(
    input_data=input_data,
    horizon_len=HORIZON_LEN,
    freq="H"
)

# Calculate MAE
def mae(pred, actual):
    return np.mean(np.abs(np.array(pred) - np.array(actual)))

base_mae = {c: mae(base_result[c], actuals[c]) for c in actuals}
print("Base Forecast MAE:")
for c, m in base_mae.items():
    print(f"  {c}: {m:.2f}")
print(f"  Average: {np.mean(list(base_mae.values())):.2f}")

---
## Forecast With Covariates

Now let's add covariates to improve accuracy.

In [ ]:
# Forecast with all covariates
cov_result = fm.forecast(
    input_data=input_data,
    horizon_len=HORIZON_LEN,
    freq="H",
    
    # Dynamic covariates
    dynamic_numerical_covariates={"gen_forecast": gen_forecast},
    dynamic_categorical_covariates={"week_day": week_day},
    
    # Static covariates
    static_categorical_covariates={"country": country_map},
    
    # XReg mode: fit linear model on covariates first, then forecast residuals
    xreg_mode="xreg + timesfm"
)

cov_mae = {c: mae(cov_result[c], actuals[c]) for c in actuals}
print("Forecast with Covariates MAE:")
for c, m in cov_mae.items():
    print(f"  {c}: {m:.2f}")
print(f"  Average: {np.mean(list(cov_mae.values())):.2f}")

In [ ]:
# Compare improvement
print("\nImprovement with Covariates:")
for c in actuals:
    improvement = (base_mae[c] - cov_mae[c]) / base_mae[c] * 100
    print(f"  {c}: {improvement:.1f}% lower MAE")

avg_improvement = (np.mean(list(base_mae.values())) - np.mean(list(cov_mae.values()))) / np.mean(list(base_mae.values())) * 100
print(f"  Average: {avg_improvement:.1f}% improvement")

In [ ]:
# Visualize the comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for ax, country in zip(axes, ['FR', 'BE']):
    x_hist = range(CONTEXT_LEN)
    x_fcst = range(CONTEXT_LEN, CONTEXT_LEN + HORIZON_LEN)
    
    # Historical
    ax.plot(x_hist, input_data[country], 'k-', alpha=0.5, label='Historical')
    
    # Actual
    ax.plot(x_fcst, actuals[country], 'go-', markersize=4, label='Actual')
    
    # Base forecast
    ax.plot(x_fcst, base_result[country], 'b--o', markersize=4, alpha=0.7, 
            label=f'Base (MAE={base_mae[country]:.1f})')
    
    # Covariate forecast
    ax.plot(x_fcst, cov_result[country], 'r--o', markersize=4, alpha=0.7,
            label=f'With Covariates (MAE={cov_mae[country]:.1f})')
    
    ax.axvline(x=CONTEXT_LEN-0.5, color='gray', linestyle='--', alpha=0.3)
    ax.set_title(f'{country} Electricity Price Forecast')
    ax.set_xlabel('Hour')
    ax.set_ylabel('Price')
    ax.legend(fontsize=9)
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

---
## XReg Modes Explained

The `xreg_mode` parameter controls how covariates are combined with time series forecasting:

| Mode | Description | Best When |
|------|-------------|----------|
| `xreg + timesfm` | Fit linear model on covariates first, forecast residuals | Covariates explain most variance |
| `timesfm + xreg` | Time series forecast first, model residuals with covariates | Strong time patterns |

In [ ]:
# Compare XReg modes
modes = ["xreg + timesfm", "timesfm + xreg"]
results = {}

for mode in modes:
    result = fm.forecast(
        input_data=input_data,
        horizon_len=HORIZON_LEN,
        freq="H",
        dynamic_numerical_covariates={"gen_forecast": gen_forecast},
        dynamic_categorical_covariates={"week_day": week_day},
        static_categorical_covariates={"country": country_map},
        xreg_mode=mode
    )
    results[mode] = result

# Compare
print("XReg Mode Comparison (Average MAE):")
for mode in modes:
    mode_mae = np.mean([mae(results[mode][c], actuals[c]) for c in actuals])
    print(f"  {mode}: {mode_mae:.2f}")

---
## Best Practices

1. **Choose relevant covariates** - Variables that have a causal relationship with your target

2. **Ensure future availability** - Dynamic covariates need known future values (forecasts, schedules)

3. **Match covariate length** - Dynamic covariates must cover `history_length + horizon_length`

4. **Experiment with XReg modes** - Try both and compare on your data

5. **Start simple** - Begin with 1-2 covariates, add more if needed

**Next:** [Batch Forecasting](03_batch_forecasting.ipynb) for production workloads